In [34]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import random_split, Dataset, DataLoader, TensorDataset
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
import numpy as np
import copy
import random

print(torch.__version__)
print(f"MPS available: {torch.backends.mps.is_available()}")
print(f"CUDA available: {torch.cuda.is_available()}")

2.2.2
MPS available: True
CUDA available: False


In [2]:
device = torch.device("mps")

In [3]:
data_dict = np.load("gt_clean_Bay5_101223.npz")

In [4]:
data_dict

NpzFile 'gt_clean_Bay5_101223.npz' with keys: ori64, sim64, ori64means, sim64means, sim64sigmas...

In [5]:
ori32 = torch.tensor(data_dict["ori64"].transpose((2,1,0))[:,:,:].astype(np.float32))
sim32 = torch.tensor(data_dict["sim64"].transpose((2,1,0))[:,:,:].astype(np.float32))
ori32means = torch.tensor(data_dict["ori64means"].transpose((2,1,0))[:,:,:].astype(np.float32))
sim32means = torch.tensor(data_dict["sim64means"].transpose((2,1,0))[:,:,:].astype(np.float32))
ori32sigmas = torch.tensor(data_dict["ori64sigmas"].transpose((2,1,0))[:,:,:].astype(np.float32))
sim32sigmas = torch.tensor(data_dict["sim64sigmas"].transpose((2,1,0))[:,:,:].astype(np.float32))

In [6]:
# create dataset from input and output
dataset = TensorDataset(ori32,sim32,ori32means,sim32means,ori32sigmas,sim32sigmas)

In [26]:
#Split dataset 80-20
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
batch_size = 8
print(train_size, test_size)
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=test_size)

1948 488


In [8]:
torch.corrcoef(torch.stack((torch.flatten(train_dataset[:][0]),torch.flatten(train_dataset[:][1])),dim=0))[1,0]

tensor(0.3658)

In [9]:
torch.corrcoef(torch.stack((torch.flatten(test_dataset[:][0]),torch.flatten(test_dataset[:][1])),dim=0))[1,0]

tensor(0.3578)

In [10]:
# torch.nn.Conv1d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros', device=None, dtype=None)
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv1d(1, 18, 9, padding="same")
        self.conv2 = nn.Conv1d(18, 18, 9, padding="same")
        self.conv3 = nn.Conv1d(18, 1, 1, padding="same")
        self.bn = nn.BatchNorm1d(18)
        self.dropout1 = nn.Dropout(0.10)
        self.dropout2 = nn.Dropout(0.5)

    def forward(self, x):
        x = self.conv1(x)
        x = F.sigmoid(x)
        
        x = self.conv2(x)
        x = self.bn(x)
        x = F.sigmoid(x)

        x = self.dropout1(x)

        x = self.conv2(x)
        x = self.bn(x)
        x = F.sigmoid(x)

        x = self.dropout1(x)

        x = self.conv2(x)
        x = self.bn(x)
        x = F.sigmoid(x)
        
        x = self.dropout1(x)

        x = self.conv2(x)
        x = self.bn(x)
        x = F.sigmoid(x)

        x = self.dropout1(x)

        x = self.conv2(x)
        x = self.bn(x)
        x = F.sigmoid(x)
        
        x = self.dropout1(x)

        x = self.conv2(x)
        x = self.bn(x)
        x = F.sigmoid(x)

        x = self.dropout1(x)
        
        output = self.conv3(x)
        
        return output

In [11]:
model = Net()

In [12]:
def custom_loss(y_true,y_pred):
    SS_res =  torch.sum(torch.square(y_true - y_pred)) 
    SS_tot = torch.sum(torch.square(y_true - torch.mean(y_true))) 
    loss2 =  (1.0 - SS_res/(SS_tot + torch.finfo(torch.float32).eps) )
    return -loss2

In [13]:
def corr_loss(y_true,y_pred):
    c = torch.corrcoef(torch.stack((torch.flatten(y_true),torch.flatten(y_pred)),dim=0))[1,0]
    return -c/(1-c)

In [31]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target,_,_,_,_) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = corr_loss(output, target)
        loss.backward()
        optimizer.step()
#        corr = corr_loss(output,target)
        if batch_idx == int(train_size/batch_size):
            print('Train Epoch: {} \t\tCorr: {:.6f}'.format(
                epoch, loss.item()/(loss.item()-1)))

In [15]:
def test(model, device, test_loader):
    model.eval()
    corr = 0
    with torch.no_grad():
        for data, target,_,_,_,_ in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            corr = corr_loss(output,target)
    c = corr.item()/(corr.item()-1)
    return c

In [16]:
model = Net().to(device)

In [42]:
best_c = [0.0]
best_lr = []
best_eps = []
best_model = copy.deepcopy(model.state_dict())

In [32]:
model = Net().to(device)
lr = 0.03
eps = 1e-8
optimizer = optim.Adam(model.parameters(),lr = lr, eps=eps)
scheduler = StepLR(optimizer, step_size=20, gamma=0.9)
for epoch in range(1, 1 + 10):
    train(model, device, train_dataloader, optimizer, epoch)
    c = test(model, device, test_dataloader)
    if c > best_c[-1]:
        print("Found better model: ",c)
        best_c.append(c)
        best_lr.append(lr)
        best_eps.append(eps)
        best_model = copy.deepcopy(model.state_dict())
    scheduler.step()

Train Epoch: 1 		Corr: 0.528292
Train Epoch: 2 		Corr: 0.023587
Train Epoch: 3 		Corr: 0.629184
Train Epoch: 4 		Corr: 0.293412
Train Epoch: 5 		Corr: 0.680213
Train Epoch: 6 		Corr: -0.050659
Train Epoch: 7 		Corr: 0.200603
Train Epoch: 8 		Corr: 0.506370
Train Epoch: 9 		Corr: 0.090546
Train Epoch: 10 		Corr: 0.470244


In [496]:
best_c[-1]

0.4434260311041125

In [44]:
# do this over and over
for lr in [0.04,0.05,0.06]:
    for eps in [0.0,1e-8,1e-7]:
        model = Net().to(device)
        print("running with lr = {:.6f} and eps = {:.6f}".format(lr,eps))
        optimizer = optim.Adam(model.parameters(),lr = lr, eps=eps)
        scheduler = StepLR(optimizer, step_size=20, gamma=0.9)
        train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=8, shuffle=True)
        for epoch in range(1, 1 + 50):
            train(model, device, train_dataloader, optimizer, epoch)
            c = test(model, device, test_dataloader)
            if c > best_c[-1]:
                print("***************** *************** Found better test model: {:.6f} ".format(c))
                best_c.append(c)
                best_lr.append(lr)
                best_eps.append(eps)
                best_model = copy.deepcopy(model.state_dict())
            scheduler.step()

running with lr = 0.040000 and eps = 0.000000
Train Epoch: 1 		Corr: 0.505193
Train Epoch: 2 		Corr: 0.107766
Train Epoch: 3 		Corr: 0.098279
Train Epoch: 4 		Corr: -0.066423
Train Epoch: 5 		Corr: 0.733172
Train Epoch: 6 		Corr: 0.324967
Train Epoch: 7 		Corr: 0.105833
Train Epoch: 8 		Corr: 0.170344
Train Epoch: 9 		Corr: 0.338722
Train Epoch: 10 		Corr: 0.705813
Train Epoch: 11 		Corr: 0.569215
Train Epoch: 12 		Corr: 0.552351
Train Epoch: 13 		Corr: -0.142048
Train Epoch: 14 		Corr: 0.047246
Train Epoch: 15 		Corr: 0.698582
Train Epoch: 16 		Corr: -0.026272
***************** *************** Found better test model: 0.410404 
Train Epoch: 17 		Corr: 0.218262
***************** *************** Found better test model: 0.419039 
Train Epoch: 18 		Corr: 0.481999
Train Epoch: 19 		Corr: 0.592886
Train Epoch: 20 		Corr: 0.540452
Train Epoch: 21 		Corr: -0.031643
Train Epoch: 22 		Corr: 0.532947
Train Epoch: 23 		Corr: 0.595526
Train Epoch: 24 		Corr: 0.248941
Train Epoch: 25 		Corr: 0.031

In [621]:
best_c[-1]

0.44217669115921787

In [622]:
best_lr

[0.060000000000000005,
 0.060000000000000005,
 0.060000000000000005,
 0.060000000000000005,
 0.060000000000000005,
 0.05,
 0.05,
 0.060000000000000005,
 0.060000000000000005,
 0.060000000000000005,
 0.060000000000000005,
 0.060000000000000005,
 0.060000000000000005,
 0.060000000000000005,
 0.04,
 0.04,
 0.04,
 0.04,
 0.04]

In [623]:
best_eps

[1e-07,
 1e-07,
 1e-07,
 1e-07,
 1e-07,
 0.0,
 0.0,
 1e-07,
 1e-07,
 1e-07,
 1e-07,
 1e-07,
 1e-07,
 1e-07,
 1e-08,
 1e-08,
 1e-08,
 1e-08,
 1e-08]

In [538]:
best_model

OrderedDict([('conv1.weight',
              tensor([[[ -7.6829,  -5.1831,  -4.6984,  11.0466,  10.7742,   1.8159,   2.5917,
                         -3.6361,   2.6898]],
              
                      [[  4.1333,   0.2688,  -2.9713,  -6.9882, -11.8796,   2.3708,   5.3326,
                         -0.9624,  -5.3388]],
              
                      [[  5.3304,   0.4318,  -2.9044,  -8.8970,   1.7081,  11.3247,  -2.7253,
                         -6.9710,  -3.2048]],
              
                      [[  4.6007,   4.3738,  -0.5227,   2.2120,   6.0605,   6.2686,  -3.8337,
                         -3.7178,  -0.6072]],
              
                      [[  2.6091,   1.5694,  -0.5482,  -4.8140,  -2.8100,  -1.3248,  -2.2218,
                         -0.8914,   4.1590]],
              
                      [[ -4.3274,   2.5563,   1.5808,  -1.7534,  -7.2680,  -5.3195,  15.1455,
                          3.3599,  -0.2593]],
              
                      [[ -2.6539,   5.78

In [624]:
torch.save(best_model, "bd_clean_Bay5model.pt")

In [625]:
# load model
model_saved = Net()
model_saved.load_state_dict(torch.load("bd_clean_Bay5model.pt", weights_only=True))
model_saved.eval()

Net(
  (conv1): Conv1d(1, 18, kernel_size=(9,), stride=(1,), padding=same)
  (conv2): Conv1d(18, 18, kernel_size=(9,), stride=(1,), padding=same)
  (conv3): Conv1d(18, 1, kernel_size=(1,), stride=(1,), padding=same)
  (bn): BatchNorm1d(18, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout1): Dropout(p=0.1, inplace=False)
  (dropout2): Dropout(p=0.5, inplace=False)
)

In [626]:
test_ori, test_sim, test_ori_mean, test_sim_mean, test_ori_sigmas, test_sim_sigmas = next(iter(test_dataloader))

In [627]:
test_cleaned = model_saved(test_ori)
test_cleaned_denorm = test_cleaned * test_ori_sigmas + test_ori_mean
test_ori_denorm = test_ori * test_ori_sigmas + test_ori_mean
test_sim_denorm = test_sim * test_sim_sigmas + test_sim_mean

In [628]:
torch.corrcoef(torch.stack((torch.flatten(test_ori),torch.flatten(test_sim)),dim=0))[1,0]

tensor(0.3657)

In [629]:
torch.corrcoef(torch.stack((torch.flatten(test_ori_denorm),torch.flatten(test_sim_denorm)),dim=0))[1,0]

tensor(0.4279)

In [630]:
torch.corrcoef(torch.stack((torch.flatten(test_cleaned),torch.flatten(test_sim)),dim=0))[1,0]

tensor(0.4422, grad_fn=<SelectBackward0>)

In [631]:
torch.corrcoef(torch.stack((torch.flatten(test_cleaned_denorm),torch.flatten(test_sim_denorm)),dim=0))[1,0]

tensor(0.5116, grad_fn=<SelectBackward0>)

In [632]:
torch.corrcoef(torch.stack((torch.flatten(dataset[:][0]),torch.flatten(dataset[:][1])),dim=0))[1,0]

tensor(0.3642)

In [633]:
torch.corrcoef(torch.stack((torch.flatten(model_saved(dataset[:][0])),torch.flatten(dataset[:][1])),dim=0))[1,0]

tensor(0.4366, grad_fn=<SelectBackward0>)

In [634]:
(0.4279-0.3657)/0.3657

0.1700847689362865